### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Group 54: Christian Sorensen, Tamunosaki Danagogo, Marcos Alvarado
> INST326
> Project 3
> 11/1/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program
>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> [Github Repo](https://github.com/chrissor-umd/INST326_Group_54)

In [140]:
# Solution - enter your code solution below
import tkinter as tk
import datetime as dt # For handling data and time calculations
from datetime import time
import json # For saving and loading data

# Stuff used for the calendar
import calendar
from IPython.display import HTML


class ScheduleManager():
    def __init__(self, day_start="00:00", day_end="23:59", shift_length_minutes=30):
        self.day_start = day_start # Earliest possible shift
        self.day_end = day_end # No more shifts will be generated past this time
        self.shift_length = shift_length_minutes # How long a shift is in minutes. When new shifts are added, the given time interval is broken down into shifts of this many minutes, which are each added to the schedule
        self.__load_json() # Loads necessary data for managing people and schedules


    #############################################################################################
    #######    SUPER AWESOME AND NOT ANNOYING JSON ENCODING STUFF I SPENT 6 HOURS ON :)    ######
    #############################################################################################

    @staticmethod
    def __json_people_hook(obj): # Converts JSON object into Person object when loading
        if "name" in obj:
            dict = obj
            name = dict["name"]
            phone = dict["phone"]
            email = dict["email"]
            person = Person(name, phone, email)
            for attr in obj.keys():
                if attr not in {'name','phone','email'}:
                    setattr(person, attr, dict[attr])
            return person
        return obj
    
    @staticmethod
    def __encode_person(obj):
        if isinstance(obj, Person):
            return obj.__dict__ # Converts a person into a serializable dictionary
        else:
            return super().default(obj)

    def __load_json(self):
        """Loads stored data from json files"""
        try:
            with open('people_data.json', 'r') as people_file:
                people_data = people_file.read()
                if len(people_data) > 0:
                    self.__people = json.loads(people_data, object_hook=self.__json_people_hook) # For some reason, I can only get this to work right by storing and loading it as a string like this
                else:
                    self.__people = {}
                print("Loaded people_data.json")
        except FileNotFoundError:
            with open('people_data.json', 'w') as people_file:
                self.__people = {}
                json.dump(self.__people, people_file, default=self.__encode_person)
                print("Created people_data.json")
        try:
            with open('shifts_data.json', 'r') as shifts_file:
                shifts = shifts_file.read()
                if len(shifts) > 0:
                    self.__shifts = json.loads(shifts)
                    print("Loaded shifts_data.json")
                else:
                    self.__shifts = self.generate_empty_shifts(day_start=self.day_start, day_end=self.day_end, shift_length=self.shift_length)
        except FileNotFoundError:
            with open('shifts_data.json', 'w') as shifts_file:
                self.__create_shifts()
                json.dump(self.__shifts, shifts_file)
                print("Created shifts_data.json")

    def __save_json(self):
        """Handles data conversions for smooth json serialization, and loads data to file"""
        with open('people_data.json', 'w') as people_file:
            json_dump = json.dumps(self.__people, indent=4, default=self.__encode_person)
            people_file.write(json_dump)
        with open('shifts_data.json', 'w') as shifts_file:
            json_dump = json.dumps(self.__shifts, indent=4)
            shifts_file.write(json_dump)
            
    ################################################
    #######         END OF JSON STUFF         ######
    ################################################


    def add_person(self, name, phone=None, email=None):
        """Creates a new person object with a unique id"""
        person_id = name.upper()
        person = Person(name, phone, email) # Creates the person object
        if person_id in self.__people.keys():
            print("Person already exists")
            return person
        person.shift_length_minutes = self.shift_length
        person.shift_times = ScheduleManager.generate_day_shift_times(self.day_start, self.day_end, self.shift_length) # Passes shift times to Person for handling availability
        person.create_availability_dict(self.shift_length, self.day_start, self.day_end)
        self.__people.update({person_id:person}) # Stores the new person in the people dictionary, with their name as the key
        print(f"Added {person.name}")
        self.__save_json()
        return person
    
    def get_person(self, name): 
        """Get the person object associated with a given name or person_id"""
        if name.upper() in self.__people:
            return self.__people[name.upper()]
        else:
            print("Person not found")

    def get_person_id(self, person_or_name): 
        """Gets a formatted person_id from either their name or a person object"""
        if isinstance(person_or_name, Person):
            person_id = person_or_name.name.upper()
        elif isinstance(person_or_name, str):
            person_id = person_or_name.upper()
            if person_id not in self.__people.keys():
                raise Exception("Person does not exist")
        return person_id

    def remove_person(self, person_id):
        """Removes person from ScheduleManager"""
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        for day in self.__shifts.keys():
            self.unassign_shift(person_id, day, '00:00', '23:59') # Removes the person from any shifts they're assigned to
        self.__people.pop(person_id) # Removes person from list
        print(f"{person.name} removed")
        self.__save_json()
        return
        
    def __create_shifts(self):
        """Restores default empty shift dictionary"""
        self.__shifts = self.generate_empty_shifts(day_start=self.day_start, day_end=self.day_end, shift_length=self.shift_length)

    def assign_shift(self, person_id, day, start, end=None):
        if end == None:
            end = self.add_time(start, self.shift_length).isoformat('minutes')
        day, time_start, time_end = self.get_formatted_shift_input(day, start, end)
        if time_start < time.fromisoformat(self.day_start) or time_end > time.fromisoformat(self.day_end):
            print("Time outside of available hours")
            return
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift_added = False
        for shift in self.__shifts[day].keys(): # Get list of shifts
            shift_start = time.fromisoformat(shift)
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within the given time interval
                if person_id not in self.__shifts[day][shift]: # Prevent duplicate entries
                    self.__shifts[day][shift].append(person_id)
                    shift_added = True
        if shift_added:
            self.__save_json()
            print(f"Added {person.name} to shifts")
        else:
            print("Could not assign shift") # In case they're already fully booked for that time frame

    def __get_default_end_time(self, start):
        """Retuns either the end of a given shift or the end of the work day, whichever happens first"""
        _, start, _ = ScheduleManager.get_formatted_shift_input('SUN', start, '23:59')
        end_time = self.add_time(start, self.shift_length)
        return end_time

    def __get_shift_from_time(self, time_to_check):
        """Gets the shift that contains a given time"""
        _, time_val, _ = ScheduleManager.get_formatted_shift_input('SUN', time_to_check, '23:59')
        if time_val < time.fromisoformat(self.day_start) or time_val > time.fromisoformat(self.day_end):
            return None
        for shift in ScheduleManager.generate_day_shift_times(self.day_start, self.day_end, self.shift_length):
            shift_start = time.fromisoformat(shift)
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_val, time_val)):
                return shift_start
            
    def unassign_shift(self, person_id, day, start, end=None):
        if end is None:
            end = self.__get_default_end_time(start) # If you only want to modify a single shift
        day, time_start, time_end = self.get_formatted_shift_input(day, start, end)
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift_removed = False
        for shift in self.__shifts[day].keys(): # Get list of shifts
            shift_start = time.fromisoformat(shift)
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within the given time interval
                if person_id in self.__shifts[day][shift]: # Prevent duplicate entries
                    self.__shifts[day][shift].remove(person_id)
                    shift_removed = True
        if shift_removed:
            self.__save_json()
            print(f"Removed {person.name} from shifts")

    def set_availability(self, person_id, availability, day, start, end=None):
        """Set availability for a specific person"""
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        if end is None:
            end = self.__get_default_end_time(start)  # If you only want to modify a single shift
            end.replace(second=0)
        person.set_shift_availability(availability, day, start, end)
        self.__save_json()

    def add_availability_exception(self, person_id, availability, date, start, end=None):
        """Add availability exception for a specific person"""
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        if end is None:
            end = self.__get_default_end_time(start)  # If you only want to modify a single shift
        person.add_shift_exception(availability, date, start, end)

    def get_availability(self, person_id, date, shift):
        """Gets availability status of a specific person on a given time and date"""
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift = self.__get_shift_from_time(shift)
        print(shift)
        return person.get_availability(date, shift)

    @staticmethod
    def add_time(base_time, minutes_to_add):
        """Datetime doesn't have a way to add time to time objects, so this handles that. Results that exceed 24 hours are capped at 23:59"""
        null_date = dt.datetime(2000, 1, 1) # Null date just so the addition works
        if isinstance(base_time, str):
            _, base_time, _ = ScheduleManager.get_formatted_shift_input('sun', base_time, '23:59') # Meaningless values, just want to do all the formatting to make sure base_time is good
        base_time = dt.datetime.combine(null_date, base_time)
        delta = dt.timedelta(minutes=minutes_to_add)
        new_datetime = base_time + delta
        new_time = new_datetime.time()
        if new_datetime.day == 1:
            new_time.replace(second=0, microsecond=0)
            return new_time
        else:
            return time(hour=23, minute=59)

    @staticmethod
    def generate_day_shift_times(day_start="00:00", day_end="23:59", shift_length=30):
        """Calculates and returns a list of shift times for a day"""
        _, day_start, day_end = ScheduleManager.get_formatted_shift_input('sun', day_start, day_end) # Only the formatted times are needed, so sunday is a placeholder
        start_shift = day_start
        current_shift = start_shift
        shift_time_list = []
        while current_shift < day_end:
            shift_time_list.append(current_shift.isoformat('minutes'))
            current_shift = ScheduleManager.add_time(current_shift, shift_length)
        return shift_time_list

    @staticmethod
    def generate_empty_shifts(day_start="00:00", day_end="23:59", shift_length=30):
        """Returns dictionary with empty shifts for each day"""
        shift_time_list = ScheduleManager.generate_day_shift_times(day_start, day_end, shift_length)
        days = {'SUN','MON','TUE','WED','THU','FRI','SAT'} # Some of this is probably redundant but I don't want any weird indexing issues
        shifts_dict = {'SUN':{},'MON':{},'TUE':{},'WED':{},'THU':{},'FRI':{},'SAT':{}} # Each day is a dictionary that will contain a day and an associated dictionary of shifts, each containing a list of scheduled people
        for day in days:
            for s in shift_time_list:
                shifts_dict[day].update({s:[]})
        print("Generated empty shifts")
        return shifts_dict

    @staticmethod
    def check_overlap(interval1, interval2):
        """Checks if there is any kind of overlap between two time intervals, and returns the type of overlap as a str, or False if no overlap is present"""
        time_list = [interval1[0],interval1[1],interval2[0],interval2[1]]
        for i, t in enumerate(time_list): # Convert each value to time. format if necessary
            if isinstance(t, str):
                try:
                    converted = time.fromisoformat(t) # Try converting time to time. format
                    time_list[i] = converted # Replace original value in list
                except ValueError:
                    raise Exception("Invalid time input") # This is just so I can have a custom error message
        int1_start = time_list[0]
        int1_end = time_list[1]
        int2_start = time_list[2]
        int2_end = time_list[3]
        if int1_start > int1_end or int2_start > int2_end: # start time should always be less than end time
            raise Exception("Invalid time input")
        if int1_end < int2_start or int1_start > int2_end: # No overlap
            return False
        elif int1_start >= int2_start and int1_end <= int2_end: # Full overlap
            return 'inside'
        elif int1_start <= int2_start and int1_end >= int2_end:
            return 'outside'
        elif int1_start > int2_start: # Left overlap
            return 'right'
        elif int1_end < int2_end: # Right overlap
            return 'left'

    @staticmethod
    def get_formatted_shift_input(day, start, end): # This is a staticmethod so any class can call it easily
        """Returns a formatted and typechecked tuple of (day, time_start, time_end), raising error on invalid input. Day can be single day as string, or list of strings"""
        if (type(day) != str and type(day) != list) or type(start) not in {str, time} or type(end) not in {str, time}:
            raise Exception("Invalid format")
        if type(day) == str:
            day = day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
            if day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
                raise Exception("Invalid day")
        elif type(day) == list:
            for d in enumerate(day):
                index = d[0]
                current_day = d[1]
                if type(current_day) != str or current_day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
                    raise Exception("Invalid day format")
                day[index] = current_day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
        try:
            if isinstance(start, str):
                time_start = time.fromisoformat(start) # Ensure that the time is in a valid convertable format
            elif isinstance(start, time):
                start = start.replace(second=0, microsecond=0)
                time_start = start # Truncates time down to minutes
            if isinstance(end, str):
                time_end = time.fromisoformat(end)
            elif isinstance(end, time):
                end = end.replace(second=0, microsecond=0)
                time_end = end # Truncates time down to minutes
        except ValueError: # If the time format cannot be converted, try messing with the string a little for easy fixes
            try:
                time_list = []
                for t in (start, end):
                    if isinstance(t, time):
                        time_list.append(t.isoformat('minutes'))
                        continue
                    tlist = t.split(":")
                    time_string = ""
                    for num in tlist:
                        if len(num) < 2: # Time conversion errors out if single digits are not prefixed with 0, so this tries to anticipate that
                            zeroes_to_add = 2-len(num)
                            num = ('0'*zeroes_to_add)+num # Prefixes zeroes
                        elif len(num) > 2:
                            num = num[0:2] # Truncate number to first 2 digits
                        time_string += f"{num}:" # Adds number to time_string, adding a colon
                    time_string = time_string[0:-1] # Remove last colon
                    time_list.append(time_string)
                time_start = time.fromisoformat(time_list[0]) # Retry conversion with modified inputs
                time_end = time.fromisoformat(time_list[1])
            except AttributeError or ValueError:
                raise Exception("Invalid time format")
        if time_start > time_end:
            raise Exception("Invalid time interval")
        for itime in (time_start, time_end): # Do some rounding and formatting for nicer and more predictable time intervals
            itime.replace(second=0, microsecond=0)
            itime.replace(minute=(itime.minute - itime.minute%5)) # Round down to the nearest interval of 5 minutes
        return (day, time_start, time_end) # Returns a tuple containing the formatted and typechecked inputs


class Person:
    def __init__(self, name, phone=None, email=None):
        self.name = str(name)
        self.phone = phone # Optional
        self.email = email # Optional
        self.__pay_rate = 0
        self.__shift_exceptions = {}
        self.shift_length_minutes = 30 # Defaults to 30 minute time intervals, should be passed specific value by ScheduleManager
        self.shift_times = [] # Default timing of shifts passed from the ScheduleManager

    def __str__(self):
        """Returns name of person when object is converted to string"""
        return str(self.name)
    
    @property
    def pay_rate(self):
        """Read Only"""
        return self.__pay_rate

    def set_pay_rate(self, new_rate):
        """Sets the new rate, checking the type and rejecting negative numbers"""
        if type(new_rate) in {float, int}:
            if new_rate >= 0:
                self.__pay_rate = new_rate
            else:
                print("New rate must be positive number or zero")
        else:
            print("Invalid pay rate amount")

    @property
    def availability(self):
        """Read only"""
        return self.__shift_availability

    def create_availability_dict(self, shift_length=None, day_start="00:00", day_end="23:59"):
        if shift_length is None:
            shift_length = self.shift_length_minutes
        days = {'SUN','MON','TUE','WED','THU','FRI','SAT'} # Some of this is probably redundant but I don't want any weird indexing issues
        self.__shift_availability = {'SUN':{},'MON':{},'TUE':{},'WED':{},'THU':{},'FRI':{},'SAT':{}} # Each day is a dictionary that will contain a day and an associated dictionary of shifts, each containing a list of scheduled people
        for day in days:
            for s in self.shift_times:
                self.__shift_availability[day].update({s:'available'})
        print(f"Created availability dictionary for {self.name}")

    def set_shift_availability(self, availability, day, start, end):
        """Changes the availability for the given day and time"""
        availability = availability.lower()
        if availability not in {'preferred', 'available', 'unavailable'}: # Check for valid availability
            print(f"Invalid availability '{availability}', please select only from: 'available', 'unavailable', 'preferred'")
            return
        day, time_start, time_end = ScheduleManager.get_formatted_shift_input(day, start, end)
        availability_dict = self.__shift_availability[day].copy()
        for shift in availability_dict.keys(): # Get list of shifts
            shift_start = time.fromisoformat(shift) # Convert to time. format for comparisons
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length_minutes)
            time_string = shift_start.isoformat('minutes')
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within the given time interval
                self.__shift_availability[day].update({time_string:availability})
        print(f"Changed availability to '{availability}' for shifts from {time_start.isoformat('minutes')} to {time_end.isoformat('minutes')}")

    def add_shift_exception(self, availability, date, start, end, recurring=False):
        """Adds an availaibility exeption for a specific date and range of time. Set recurring to True for it to apply for every year"""
        if availability not in {'preferred', 'available', 'unavailable'}: # Check for valid availability
            print(f"Invalid availability '{availability}', please select only from: 'available', 'unavailable', 'preferred'")
            return
        if isinstance(date, str):
            date = dt.date.fromisoformat(date)
        elif isinstance(date, dt.datetime):
            date = date.date()
        else:
            print("Invalid date")
            return
        _, time_start, time_end = ScheduleManager.get_formatted_shift_input('SUN', start, end)
        if date.isoformat() not in self.__shift_exceptions:
            self.__shift_exceptions.update({date.isoformat():{}})
        date_shift_exceptions = self.__shift_exceptions[date.isoformat()]
        preference_changed = False
        for shift in self.shift_times:
            shift_start = time.fromisoformat(shift)
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length_minutes)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within given time range
                date_shift_exceptions.update({shift:[availability, recurring]})
                preference_changed = True
        if preference_changed:
            print("Changed preferences")
        else:
            if not self.__shift_exceptions[date.isoformat()]:
                self.__shift_exceptions.pop(date.isoformat())

    def update_exceptions(self):
        """Updates all recurring shift exceptions to apply to the current year, and cleans old shift exceptions"""
        current_year = dt.datetime.now().year
        for date_index in self.__shift_exceptions.keys():
            date_exceptions = self.__shift_exceptions[date_index]
            for shift in date_exceptions.keys():
                status_list = date_exceptions[shift]
                date = dt.date.fromisoformat(date_index)
                recurring = status_list[1] # Check if exception is recurring
                if date.year < current_year:
                    if recurring:
                        self.__shift_exceptions.pop(date_index) # Remove old entry
                        date.replace(year=current_year) # Updates year
                        date_index = date.isoformat()
                        self.__shift_exceptions.update({date_index:[status_list]}) # Replaces old entry with new one
                    else:
                        self.__shift_exceptions.pop(date_index) # Removes old entry
        
    def get_availability(self, date_, shift):
        self.update_exceptions()
        if isinstance(date_, str):
            try:
                date_ = dt.date.fromisoformat(date_) # Ensure propper formatting
                date_str = date_.isoformat()
            except ValueError:
                print("Invalid date format, please use 'YYYY-MM-DD'")
                return
        elif isinstance(date_, dt.date):
            date_str = date_.isoformat()
        elif isinstance(date_, dt.datetime):
            date_str = date_.date().isoformat()
            date_ = date_.date()
        else:
            print("Invalid date")
            return
        _, shift_time, _ = ScheduleManager.get_formatted_shift_input('SUN', shift, '23:59')
        shift_str = shift_time.isoformat('minutes')
        if shift_str not in self.shift_times: # Check if the shift availability needs to be calculated
            work_end = ScheduleManager.add_time(self.shift_times[-1], self.shift_length_minutes) # Calculate when the last shift would end
            if shift_time < time.fromisoformat(self.shift_times[0]) or shift_time > work_end:
                return 'unavailable'
            for i, t in enumerate(self.shift_times): # If the given time is not a shift starting time, calculate availability based on which shift the given time is in
                if i < len(self.shift_times)-1: # Check to avoid exceeding the list length
                    current_shift = time.fromisoformat(self.shift_times[i])
                    next_shift = time.fromisoformat(self.shift_times[i+1])
                    if current_shift <= shift_time < next_shift: # Check between which two shifts the given time falls under
                            
                            shift_time = current_shift
                            shift_str = shift_time.isoformat('minutes')
                            break
                else:
                    last_shift = time.fromisoformat(self.shift_times[-1])
                    if last_shift <= shift_time < work_end:
                        shift_time = last_shift
                        shift_str = shift_time.isoformat('minutes')
                    else:
                        return 'unavailable'
        if date_str in self.__shift_exceptions.keys() and shift_str in self.__shift_exceptions[date_str].keys(): # Check if the date has an availability exception
            availability_dict = self.__shift_exceptions[date_str]
            return availability_dict[shift_str][0]
        else:
            days = ['MON','TUE','WED','THU','FRI','SAT','SUN'] # date.weekday() has 0 set as Monday, for some reason
            day = days[date_.weekday()] # Gets the day of the week from the given date
            day, _, _ = ScheduleManager.get_formatted_shift_input(day, '00:00','23:59')
            availability_dict = self.__shift_availability[day]
            return availability_dict[shift_str]
                
#Calendar stuff

class CalendarManager:
    def __init__(self):
        # Initialize HTMLCalendar
        html_cal = calendar.HTMLCalendar()
        year = 2024

        # creating the HTML content for the year 2024 calendar
        calendar_html = html_cal.formatyear(year)

        # Display the calendar as HTML in the notebook
        HTML(calendar_html)


if __name__ == "__main__":
    schedule = ScheduleManager('07:00', '19:00', 360)
    person1 = schedule.add_person('Joe')
    person2 = schedule.add_person('Bob')
    schedule.remove_person('Bob')
    schedule.assign_shift(person1, 'sun', '10:30','17:00')
    schedule.unassign_shift(person1, 'sun', '10:30')
    schedule.add_availability_exception(person1, 'preferred', '2024-11-20', '10:00','12:00')
    schedule.set_availability(person1, 'preferred', 'sun', '5:30')
    now = dt.datetime.now()
    now_date = now.date()
    now_time = now.time()
    print(schedule.get_availability(person1, now_date, now_time))

Loaded people_data.json
Loaded shifts_data.json
Person already exists
Created availability dictionary for Bob
Added Bob
Bob removed
Added Joe to shifts
Removed Joe from shifts
Changed preferences
Changed availability to 'preferred' for shifts from 05:30 to 11:30
13:00:00
available


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS